In [ ]:
!pip install flexible-fl opacus SciencePlots setuptools flexclash

In [ ]:
import copy
import os
import math
import torch
from flex.data import Dataset, FedDataDistribution, FedDataset, FedDatasetConfig
from flex.model import FlexModel
from flex.pool import FlexPool, fed_avg
from flex.pool.decorators import (
    deploy_server_model,
    init_server_model,
    set_aggregated_weights,
    collect_clients_weights,
)
from flexclash.data import data_poisoner
from flexclash.pool.defences import central_differential_privacy
import matplotlib.pyplot as plt
import matplotlib as mpl
import scienceplots
from typing import List
import numpy as np
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader
import torchvision.models as models
from torchvision import transforms
from torchvision.datasets import CIFAR10, FashionMNIST
import opacus
from opacus import PrivacyEngine
from opacus.validators import ModuleValidator
from opacus.accountants.utils import get_noise_multiplier
from scipy.optimize import linprog
import pandas as pd
from PIL import Image

# --- CONSTANTS ---
ROUNDS = 100
EPOCHS = 1
N_NODES = 10
POISONED = 2
epsilon = np.full(ROUNDS, 1.0)
delta = np.full(ROUNDS, 0.001)
budget = 100.0

device = "cuda" if torch.cuda.is_available() else "cpu"

fashion_transforms = transforms.Compose(
    [
        transforms.ToTensor(),
    ]
)

In [ ]:
def get_dataset():
    """
    Returns a FlexDataset object containing FashionMNIST data.
    """
    train_data = FashionMNIST(root=".", train=True, download=True, transform=None)
    test_data = FashionMNIST(root=".", train=False, download=True, transform=None)
    flex_dataset = Dataset.from_torchvision_dataset(train_data)
    test_data = Dataset.from_torchvision_dataset(test_data)
    assert isinstance(flex_dataset, Dataset)

    config = FedDatasetConfig(seed=0)
    config.replacement = False
    config.n_nodes = N_NODES

    flex_dataset = FedDataDistribution.from_config(flex_dataset, config)

    assert isinstance(flex_dataset, FedDataset)
    flex_dataset["server"] = test_data

    return flex_dataset

In [ ]:
class CNNModel(nn.Module):
    def __init__(self, num_classes):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=5)
        self.pool = nn.MaxPool2d(kernel_size=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=5)
        self.fc1 = nn.Linear(1024, 200)
        self.fc2 = nn.Linear(200, num_classes)

    def forward(self, x):
        x = self.pool(torch.tanh(self.conv1(x)))
        x = self.pool(torch.tanh(self.conv2(x)))
        x = x.view(-1, 1024)
        x = torch.tanh(self.fc1(x))
        x = self.fc2(x)
        return x

def get_model(num_classes=10):
  return ModuleValidator.fix(CNNModel(num_classes=num_classes))

In [ ]:
# FLEX Decorators
@init_server_model
def build_server_model():
    server_flex_model = FlexModel()
    server_flex_model["model"] = get_model()
    server_flex_model["criterion"] = torch.nn.CrossEntropyLoss()
    server_flex_model["optimizer_func"] = torch.optim.Adam
    server_flex_model["optimizer_kwargs"] = {}
    return server_flex_model

@deploy_server_model
def copy_server_model_to_clients(server_flex_model: FlexModel):
    new_flex_model = FlexModel()
    new_flex_model["model"] = copy.deepcopy(server_flex_model["model"])
    new_flex_model["server_model"] = copy.deepcopy(server_flex_model["model"])
    new_flex_model["discriminator"] = copy.deepcopy(server_flex_model["model"])
    new_flex_model["criterion"] = copy.deepcopy(server_flex_model["criterion"])
    new_flex_model["optimizer_func"] = copy.deepcopy(
        server_flex_model["optimizer_func"]
    )
    new_flex_model["optimizer_kwargs"] = copy.deepcopy(
        server_flex_model["optimizer_kwargs"]
    )
    return new_flex_model

@set_aggregated_weights
def set_agreggated_weights_to_server(server_flex_model: FlexModel, aggregated_weights):
    dev = aggregated_weights[0].get_device()
    dev = "cpu" if dev == -1 else "cuda"
    with torch.no_grad():
        weight_dict = server_flex_model["model"].state_dict()
        for layer_key, new in zip(weight_dict, aggregated_weights):
            weight_dict[layer_key].copy_(weight_dict[layer_key].to(dev) + new)

@collect_clients_weights
def get_clients_weights(client_flex_model: FlexModel):
    weight_dict = client_flex_model["model"].state_dict()
    server_dict = client_flex_model["server_model"].state_dict()
    dev = [weight_dict[name] for name in weight_dict][0].get_device()
    dev = "cpu" if dev == -1 else "cuda"
    return [
        (weight_dict[name] - server_dict[name].to(dev)).type(torch.float)
        for name in weight_dict
    ]

In [ ]:
def train(client_flex_model: FlexModel, client_data: Dataset):
    """
    Train the model on the client data.
    """
    model = client_flex_model["model"]
    criterion = client_flex_model["criterion"]
    model.train()
    model = model.to(device)
    torch_dataset = client_data.to_torchvision_dataset(transform=fashion_transforms)
    optimizer = client_flex_model["optimizer_func"](model.parameters(), **client_flex_model["optimizer_kwargs"])
    dataloader = DataLoader(
        torch_dataset, batch_size=32, shuffle=True, pin_memory=False
    )

    for _ in range(EPOCHS):
        running_loss = 0.0
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

    return running_loss

def evaluate_model(server_flex_model: FlexModel, data):
    """
    Evaluate the model on the server data.
    """
    data = flex_dataset["server"]
    model = server_flex_model["model"]
    model.eval()
    test_loss = 0
    test_acc = 0
    total_count = 0
    model = model.to(device)
    criterion = server_flex_model["criterion"]

    test_dataset = data.to_torchvision_dataset(transform=fashion_transforms)
    test_dataloader = DataLoader(
        test_dataset, batch_size=32, shuffle=True, pin_memory=False
    )
    losses = []
    with torch.no_grad():
        for data, target in test_dataloader:
            total_count += target.size(0)
            data, target = data.to(device), target.to(device)
            output = model(data)
            losses.append(criterion(output, target).item())
            pred = output.data.max(1, keepdim=True)[1]
            test_acc += pred.eq(target.data.view_as(pred)).long().cpu().sum().item()

    test_loss = sum(losses) / len(losses)
    test_acc /= total_count
    return test_loss, test_acc

In [ ]:
@data_poisoner
def backdoor_cross(img, label, prob=0.3, target_label=4):
    """
    Apply a backdoor to the image.
    """
    if np.random.random() > prob:
        return img, label

    arr = np.array(img)
    new_arr = copy.deepcopy(arr)

    if not new_arr.flags.writeable:
        new_arr = new_arr.copy()

    size = 5
    cx, cy = 2, 2

    new_arr[0:size, -size:] = 255

    for i in range(size):
        new_arr[i, -size + cx] = 0
        new_arr[cy, -size + i] = 0

    return Image.fromarray(new_arr), target_label

def backdoor_cross_no_decorator(img, label, prob=1.0, target_label=4):
    """
    Apply a backdoor to the image. All images are backdoored (to test the ASR)
    """
    if np.random.random() > prob:
        return img, label

    arr = np.array(img)
    new_arr = copy.deepcopy(arr)

    if not new_arr.flags.writeable:
        new_arr = new_arr.copy()

    size = 5
    cx, cy = 2, 2
    new_arr[0:size, -size:] = 255

    for i in range(size):
        new_arr[i, -size + cx] = 0
        new_arr[cy, -size + i] = 0

    return Image.fromarray(new_arr), target_label

def generate_backdoored_test_set(server_flex_model: FlexModel, data, target_label=4, prob=1.0):
    """
    Generate a backdoored test set.
    """
    data = flex_dataset["server"]
    model = server_flex_model["model"]
    model.eval()
    backdoored_images = []
    backdoored_labels = []

    for img, label in data:
        triggered_img, _ = backdoor_cross_no_decorator(img, label, prob=prob, target_label=target_label)
        backdoored_images.append(fashion_transforms(triggered_img))
        backdoored_labels.append(label)

    backdoored_images = torch.stack(backdoored_images)
    backdoored_labels = torch.tensor(backdoored_labels)
    return backdoored_images, backdoored_labels, model

def compute_ASR(model, backdoored_images, target_label, device):
    """
    Compute the ASR of the model, so that we can evaluate the backdoor attack.
    """
    model.eval()
    model.to(device)
    print(target_label)
    with torch.no_grad():
        outputs = model(backdoored_images.to(device))
        preds = torch.argmax(outputs, dim=1)
        print("predicciones:", preds)
        successful = (preds == target_label).sum().item()
        return successful / len(backdoored_images)

In [ ]:
def optimize_epsilon_delta_one_round(loss_t, loss_min, epsilon_t, delta_t, sensitivity, total_budget, used_budget, gamma):
    """
    Optimize the epsilon and delta parameters for the next round.
    """
    delta_loss = loss_t - loss_min

    # We define these weights, depending on the loss
    alpha = 1 + max(0, delta_loss)
    beta = 1 - min(0, delta_loss)

    eps_lower_val = max(0.1, epsilon_t - 0.1)
    eps_upper_val = min(1.5, epsilon_t + 0.1)
    sigma_lower = sensitivity * math.sqrt(2 * math.log(1.25 / delta_t)) / eps_lower_val
    sigma_upper = sensitivity * math.sqrt(2 * math.log(1.25 / delta_t)) / eps_upper_val

    # Definition of the coefficient for the lineal relationship sigma = a*epsilon +b
    a = (sigma_upper - sigma_lower) / (eps_upper_val - eps_lower_val)
    b = sigma_lower - a * eps_lower_val

    # If the loss is greater, we prefer greater epsilon
    if delta_loss > 0:
        c = [alpha + gamma * a + 1.5, beta + 1.5]
    else:
        c = [- (alpha + gamma * a - 1.5), - (beta - 1.5)]

    effective_budget = total_budget - used_budget

    A = [[1, 1]]
    b_ub = [effective_budget]

    delta_eps_lower = max(-0.2, -epsilon_t + 0.5)
    delta_eps_upper = 1.0
    delta_delta_lower = max(-1e-7, -delta_t + 0.0001)
    delta_delta_upper = 1e-7

    bounds = [(delta_eps_lower, delta_eps_upper), (delta_delta_lower, delta_delta_upper)]

    # Solve the LP Problem
    res = linprog(c, A_ub=A, b_ub=b_ub, bounds=bounds, method='simplex')

    if res.success:
        delta_epsilon, delta_delta = res.x
        new_epsilon = epsilon_t + delta_epsilon
        new_delta = delta_t + delta_delta
    else:
        new_epsilon, new_delta = epsilon_t, delta_t

    return new_epsilon, new_delta

In [ ]:
def run_attack_optimize_DP(pool: FlexPool):
    """
    Run the attack and optimize the epsilon and delta parameters.
    """

    clients = pool.clients
    server = pool.servers

    epsilon_used = 0
    losses = []
    accuracies=[]
    epsilon_cummulative = []
    asr_over_rounds = []

    for i in range(ROUNDS):

        print(f"\n - Round {i+1}: Aggregating with with ε={epsilon[i]:.3f}, δ={delta[i]:.5f}")
        server.map(copy_server_model_to_clients, clients)


        epsilon_used += epsilon[i]
        loss = clients.map(train)
        losses.append(loss[0])

        sensitivity = 0.01
        noise_multiplier = (sensitivity / epsilon[i]) * np.sqrt(2 * np.log(1.25 / delta[i]))
        print("Noise multiplier", noise_multiplier)

        pool.servers.map(get_clients_weights, clients)
        pool.servers.map(central_differential_privacy, l2_clip = 1.0, noise_multiplier = noise_multiplier)
        pool.servers.map(set_agreggated_weights_to_server, pool.servers)

        if i >= 0 and i < ROUNDS-1:
          epsilon[i+1], delta[i+1] = optimize_epsilon_delta_one_round(loss[0], losses[i-1], epsilon[i], delta[i], 1.0, budget, epsilon_used, 0.5)

        round_metrics = pool.servers.map(evaluate_model)
        accuracies.append(round_metrics[0][1]*100)
        print(" * Round metrics: ", round_metrics)
        epsilon_cummulative.append(epsilon_used)
        print(f"Epsilon used: {epsilon_used} \n")

        backdoored_test_set = pool.servers.map(generate_backdoored_test_set, target_label=4, prob=1.0)
        backdoor_imgs = backdoored_test_set[0][0]
        model = backdoored_test_set[0][2]
        asr = compute_ASR(model, backdoor_imgs, target_label=4, device=device)
        asr_over_rounds.append(asr)
        print(f"ASR en ronda {i+1}: {asr*100:.2f}%")

        with open("experiment_DP_LP.txt", "a") as archivo:
            archivo.write(f"\n - Round {i+1}: Training with ε={epsilon[i]:.3f}, δ={delta[i]:.5f}\n")
            archivo.write(f"Round metrics: {round_metrics}\n")
            archivo.write(f"Epsilon used: {epsilon_used}\n")
            archivo.write(f"ASR: {asr}\n")
            archivo.write("-" * 30 + "\n")

    df_metrics = pd.DataFrame({
      'Round': list(range(1, ROUNDS + 1)),
      'Accuracy (%)': accuracies,
      'Epsilon Acumulado': epsilon_cummulative,
      'ASR': asr_over_rounds
      })
    df_metrics.to_csv('metrics_DP_LP.csv', index=False)

In [ ]:
if __name__ == "__main__":
    flex_dataset = get_dataset()
    poisoned_clients_ids = list(flex_dataset.keys())[:POISONED]
    flex_dataset = flex_dataset.apply(backdoor_cross, node_ids=poisoned_clients_ids)
    pool = FlexPool.client_server_pool(
        fed_dataset=flex_dataset, init_func=build_server_model
    )
    run_attack_optimize_DP(pool)
    for client in poisoned_clients_ids:
      poisoned_dataset = flex_dataset[client]
      fig, ax = plt.subplots(1,1)
      for x, y in poisoned_dataset[3:]:
          ax.set_title(f"Sample from poisoned client {client}, label {y}")
          ax.axis('off')
          ax.imshow(x, cmap=plt.get_cmap('gray'))
          break
      plt.show()